In [1]:
%run imports.py

In [ ]:
store = DataStore('/data/tabular_data')
dataset = store.load_dataset('adult', 0)
print(dataset.task_type)

metric = {
    TaskType.REGRESSION: 'd2_absolute_error',
    TaskType.BINARY: 'neg_log_loss',
    TaskType.MULTICLASS: 'neg_log_loss',
}[dataset.task_type]

pd.concat([dataset.y.rename('TARGET'), dataset.X], axis='columns')

In [3]:
class Preproc(AbstractPreprocConstructor):
    def get_num_transform(self) -> PreprocType | Literal['skip']:
        return PreprocType.STANDARD
    def get_cat_transform(self) -> PreprocType | Literal['skip', 'auto']:
        return 'skip'

preproc = Preproc().get_preproc(dataset)

model = LinearCV(
    task_type=dataset.task_type,
    reg_type='l2',
    preproc=preproc,
    y_transform='standard',
)

model.fit(dataset.X_train, dataset.y_train)

In [4]:
subsets = ['train', 'test']
if 'val' in dataset.iloc:
    subsets.append('val')

scores = default_scoring(
    Predictions(model.predict(dataset.X), dataset=dataset),
    subsets=subsets,
)

cross_val_preds = Predictions(
    model.cross_val_predict(dataset.X_trainval, dataset.y_trainval),
    dataset.y_trainval,
    dataset.task_type,
    iloc={'cross_val': np.arange(len(dataset.y_trainval))},
)
cross_val_scores = default_scoring(
    cross_val_preds, subsets=['cross_val']
)

scores=Scores.concat([scores, cross_val_scores])

print(metric)
print(scores.bootstrap_quantiles('test', metric, (0.1, 0.5, 0.9)))
print(scores.bootstrap_quantiles('cross_val', metric, (0.1, 0.5, 0.9)))

neg_log_loss
[-0.4205322  -0.41336508 -0.40262261]
[-0.40961409 -0.40594572 -0.40296785]
